In [ ]:
# imports
import numpy as np
import os
import matplotlib.pyplot as plt
from math import sqrt
from rbtree import RedBlackTree
from IPython.display import Image

# allows plots to display in the notebook instead of another window
%matplotlib inline

In [ ]:
# configurations
rxlo = 914  # center frequency
rxfs = 40  # sampling rate in megasamples/second
rxbw = 40  # bw in MHz
buf_size = (rxfs * 1e6)/10  # 100 ms of a full second of samples
psd_len = 0  # a power of 2 that holds spectrum obtain from welch's method

f = np.linspace(-(rxfs/2), (rxfs/2), psd_len)  # frequency in Hz
f = (f + rxlo) / 1e6

threshold = 0  # threshold signals need to meet to be considered relevant

sets = np.emtpy(())
mean = np.emtpy(())

freq_tree = RedBlackTree()

In [ ]:
# get data from iq files
path = str(os.path.dirname(os.path.realpath(__file__)))
dir_list = os.listdir(path)

A red black tree is used to find new frequencies and compare them to existing ones in O(nlogn). For future considerations, everytime a frequency is matched in the RBTree, readjust the key value by averaging. This should be paired with a script that uses and ADALM-PLUTO device that collects samples and saves them to iq data points.

As of right now, the bottom code needs to be refined and will not currently work, but the basic concepts are still there. 

Link explaining more about the ztest and some formulas...
http://homework.uoregon.edu/pub/class/es202/ztest.html

![title](assets/ztest_overview.png)

In [ ]:
# start comparing signals from iq files and use z test to place them in the right node
for i in range(len(dir_list)):
    file = dir_list[i]
    sets[i] = np.fromfile(path + file, np.float32)
    mean[i] = f[np.argmax(sets[i])]
    indices = np.where(sets[i] > threshold)[0]
    num_datapts[i] = len(indices)
    sum = 0
    if len(indices) > 0:
        for j in range(len(indices)):
            idx = indicies[j]
            sum += abs(mean[i] - f[idx])**2
        std_dev[i] = sqrt(sum/len(indices))
        if i > 0:
            x1 = mean[0]
            x2 = mean[i]
            omega1 = std_dev[0]/sqrt(num_datapts[0])
            omega2 = std_dev[i]/sqrt(num_datapts[i])
            z_vals[i-1] = (abs(x1-x2)/sqrt(omega1 ** 2 + omega2 ** 2))